This notebook cleans DB23 (`ncdd_admin_database_25provinces__2023.xlsx` from Lok on 20231212), processing it into a village-level CSV with commune, district, and province info.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from itables import show

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from utils.clean_DB23 import *

In [4]:
# paths
code_path = Path().absolute()
datafd_path = code_path.parent.parent / 'data'
intmd_outfd_path = code_path.parent.parent / 'output_intmd' / 'match_CF_DB23'

### Read in DB23 and stack all sheets vertically into `db_df`

In [5]:
DB23_path = datafd_path / 'other' / 'ncdd_admin_database_25provinces__2023.xlsx'
DB23_df_dic = pd.read_excel(DB23_path, sheet_name=None, header=2)  
# all sheets, key=sheet name, skip first 2 rows, make row 3 header

In [6]:
# List of sheet names
sht_name_lst = list(DB23_df_dic.keys())
# List of sheet df
db_prov_df_lst = [DB23_df_dic[sht] for sht in sht_name_lst]

In [7]:
# Checks

# prov1_df = db_prov_df_lst[0]
# prov1_columns = prov1_df.columns
# nrow = prov1_df.shape[0]
# for df in db_prov_df_lst[1:]:
#     assert (df.columns == prov1_columns).all()
#     print(df.shape)
#     nrow = nrow + df.shape[0]
# nrow

In [8]:
# Insert province info as first row in each sheet df
colnames = db_prov_df_lst[0].columns
for i, df in enumerate(db_prov_df_lst):
    sht_name = sht_name_lst[i]
    prov_code = int(sht_name[:2])
    prov_name = sht_name[4:]
    prov_row = dict(zip(
        colnames, ['Province', prov_code, np.nan, prov_name, np.nan, np.nan]
    ))
    prov_row_df = pd.DataFrame([prov_row])
    db_prov_df_lst[i] = pd.concat([prov_row_df, db_prov_df_lst[i]]).reset_index(drop=True)
db_prov_df_lst[0].head(2)

,Type,Code,Name (Khmer),Name (Latin),Reference,Official Note,Note (by Checker)
0,Province,1,NaN,Banteay Meanchey,NaN,NaN,NaN
1,ស្រុក,102,មង្គលបូរី,Mongkol Borei,ប្រកាសលេខ ៤៩៣ប្រ.ក,NaN,NaN


In [9]:
# Stack all dfs vertically
db_df = pd.concat(db_prov_df_lst)

In [10]:
# db_df.to_csv(
#     datafd_path / 'other' / 'ncdd_admin_database_25provinces__2023_stacked.csv'
# )                        

### Checks before transformation

In [11]:
db_df.dtypes  # Code is int64

Type                  object
Code                   int64
Name (Khmer)          object
Name (Latin)          object
Reference             object
Official Note         object
Note (by Checker)    float64
dtype: object

In [12]:
db_df.isna().sum()  # First 4 columns have no unexpected missing values
# Note (by Checker) all missing

Type                     0
Code                     0
Name (Khmer)            25
Name (Latin)             0
Reference              161
Official Note        12753
Note (by Checker)    16449
dtype: int64

In [13]:
db_df.Type.unique()
# Province, District, Commune, Village, Capital District, Special Commune, Special District

array(['Province', 'ស្រុក', 'ឃុំ', 'ភូមិ', 'ក្រុង', 'សង្កាត់', 'ខណ្ឌ'],
      dtype=object)

In [14]:
# Code lengths are as expected

# Let's analyze the format of the 'Code' column for each 'Type'
code_formats = db_df.groupby('Type')['Code'].apply(lambda x: x.astype(str).apply(len).unique())
code_formats_dict = code_formats.to_dict()

# Summarizing the formats for each type
code_format_summary = {type_: f"Lengths: {lengths}" for type_, lengths in code_formats_dict.items()}
code_format_summary

{'Province': 'Lengths: [1 2]',
 'ក្រុង': 'Lengths: [3 4]',
 'ខណ្ឌ': 'Lengths: [4]',
 'ឃុំ': 'Lengths: [5 6]',
 'ភូមិ': 'Lengths: [7 8]',
 'សង្កាត់': 'Lengths: [5 6]',
 'ស្រុក': 'Lengths: [3 4]'}

In [15]:
# To confirm the proposed code structure "PPDDCCVV" in the dataset, we will analyze the codes of each type.
# We expect the following:
# - Province codes to be 1 or 2 digits (P or PP).
# - District codes to be 3 or 4 digits (PDD or PPDD) with DD between 0 and 20.
# - Commune codes to be 5 or 6 digits (PPDDCC or PDDCC) with CC between 0 and 30.
# - Village codes to be 7 or 8 digits (PPDDCCVV or PDDCCVV) with VV between 0 and 35.

# Apply the check to the dataset
code_structure_check = db_df.apply(lambda x: check_code_structure(x['Code'], x['Type']), axis=1)

# Calculate the proportion of rows that match the expected pattern
matching_pattern_proportion = code_structure_check.mean()

matching_pattern_proportion

1.0

In [16]:
db_df.Code.duplicated().any()  # No duplicate code

False

### Transformation

Goal: Create a dataset where each row represents a village with details on its associated commune, district, and province (including code, Latin name, reference, official note).

Method: we will traverse the stacked `db_df` by row. As we identify each village, we'll add a new row to our new dataset, including info about the village and its province, district, and commune.

In [17]:
village_data = []

# Iterating through the rows in the dataset
for index, row in db_df.iterrows():
    if row['Type'] == 'Province':
        current_province_code = row['Code']
        current_province_name = row['Name (Latin)']
        current_province_ref = row['Reference']
        current_province_note = row['Official Note']
    elif row['Type'] in ['ស្រុក', 'ក្រុង', 'ខណ្ឌ']:  # District or equivalent
        current_district_code = row['Code']
        current_district_name = row['Name (Latin)']
        current_district_ref = row['Reference']
        current_district_note = row['Official Note']
        assert str(current_district_code)[:-2] == str(current_province_code)
    elif row['Type'] in ['ឃុំ', 'សង្កាត់']:  # Commune or equivalent
        current_commune_code = row['Code']
        current_commune_name = row['Name (Latin)']
        current_commune_ref = row['Reference']
        current_commune_note = row['Official Note']
        assert str(current_commune_code)[:-2] == str(current_district_code)
    elif row['Type'] == 'ភូមិ':  # Village
        village_info = {
            'Village Code': row['Code'],
            'Village Name': row['Name (Latin)'],
            'Village Reference': row['Reference'],
            'Village Note': row['Official Note'],
            'Province Code': current_province_code,
            'Province Name': current_province_name,
            'Province Reference': current_province_ref,
            'Province Note': current_province_note,
            'District Code': current_district_code,
            'District Name': current_district_name,
            'District Reference': current_district_ref,
            'District Note': current_district_note,
            'Commune Code': current_commune_code,
            'Commune Name': current_commune_name,
            'Commune Reference': current_commune_ref,
            'Commune Note': current_commune_note
        }
        assert str(village_info['Village Code'])[:-2] == str(current_commune_code)
        village_data.append(village_info)
    else:
        print('error')

# Creating the DataFrame from the list
village_data = pd.DataFrame(village_data)
village_data.head()

,Village Code,Village Name,Village Reference,Village Note,Province Code,Province Name,Province Reference,Province Note,District Code,District Name,District Reference,District Note,Commune Code,Commune Name,Commune Reference,Commune Note
0,1020101,Ou Thum,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN,1,Banteay Meanchey,NaN,NaN,102,Mongkol Borei,ប្រកាសលេខ ៤៩៣ប្រ.ក,NaN,10201,Banteay Neang,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN
1,1020102,Phnum,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN,1,Banteay Meanchey,NaN,NaN,102,Mongkol Borei,ប្រកាសលេខ ៤៩៣ប្រ.ក,NaN,10201,Banteay Neang,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN
2,1020103,Banteay Neang,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN,1,Banteay Meanchey,NaN,NaN,102,Mongkol Borei,ប្រកាសលេខ ៤៩៣ប្រ.ក,NaN,10201,Banteay Neang,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN
3,1020104,Kouk Pnov,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN,1,Banteay Meanchey,NaN,NaN,102,Mongkol Borei,ប្រកាសលេខ ៤៩៣ប្រ.ក,NaN,10201,Banteay Neang,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN
4,1020105,Trang,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN,1,Banteay Meanchey,NaN,NaN,102,Mongkol Borei,ប្រកាសលេខ ៤៩៣ប្រ.ក,NaN,10201,Banteay Neang,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN


In [18]:
# village_data.to_csv(
#     intmd_outfd_path / 'ncdd_admin_database_25provinces__2023_wide_en.csv'
# )   